In [1]:

from tensorflow import keras
from sklearn.model_selection import train_test_split
import pickle
import os
import psutil

In [2]:
def carregar_pickle(nome_arquivo):
  with open(nome_arquivo, 'rb') as arquivo:
    objeto = pickle.load(arquivo)
  return objeto


def process_memory():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss

def profile(func):
    def wrapper(*args, **kwargs):

        mem_before = process_memory()
        result = func(*args, **kwargs)
        mem_after = process_memory()
        print('function: '+ func.__name__ + ' | '+ 'memo after: ' + str(mem_after) 
              + ' | ' + 'memo before: ' + str(mem_before) + ' | '+ 'memo diff:' + str(mem_after - mem_before))

        return result
    return wrapper    

@profile
def trainning_function(input_train, output_train, input_validation, output_validation):

    uniqueInput = keras.Input(shape=(497, 16)) 

    conv1R0 = keras.layers.Conv1D(10, kernel_size= 1, strides= 1, padding='same', activation='relu', input_shape = (497, 16), use_bias= True)(uniqueInput)
    pool1R0 = keras.layers.AveragePooling1D(pool_size=11, strides= 3, padding='valid')(conv1R0)
    conv2R0 = keras.layers.Conv1D(35, kernel_size= 1, strides= 1, padding='valid', activation='relu', input_shape = (163, 10), use_bias= True)(pool1R0)
    pool2R0 = keras.layers.AveragePooling1D(pool_size=37, strides= 21, padding='valid')(conv2R0)
    conv3R0 = keras.layers.Conv1D(60, kernel_size= 1, strides= 1, padding='valid', activation='relu', input_shape = (35, 7), use_bias= True)(pool2R0)
    pool3R0 = keras.layers.GlobalAveragePooling1D()(conv3R0)
    flat0 =  keras.layers.Flatten()(pool3R0)

    conv1R1 = keras.layers.Conv1D(10, kernel_size= 3, strides= 1, padding='same', activation='relu', input_shape = (497, 16), use_bias= True)(uniqueInput)
    pool1R1 = keras.layers.AveragePooling1D(pool_size=11, strides= 3, padding='valid')(conv1R1)
    conv2R1 = keras.layers.Conv1D(35, kernel_size= 1, strides= 1, padding='valid', activation='relu', input_shape = (163, 10), use_bias= True)(pool1R1)
    pool2R1 = keras.layers.AveragePooling1D(pool_size=37, strides= 21, padding='valid')(conv2R1)
    conv3R1 = keras.layers.Conv1D(60, kernel_size= 1, strides= 1, padding='valid', activation='relu', input_shape = (35, 7), use_bias= True)(pool2R1)
    pool3R1 = keras.layers.GlobalAveragePooling1D()(conv3R1)
    flat1 =  keras.layers.Flatten()(pool3R1)

    conv1R2 = keras.layers.Conv1D(10, kernel_size= 6, strides= 1, padding='same', activation='relu', input_shape = (497, 16), use_bias= True)(uniqueInput)
    pool1R2 = keras.layers.AveragePooling1D(pool_size=11, strides= 3, padding='valid')(conv1R2)
    conv2R2 = keras.layers.Conv1D(35, kernel_size= 1, strides= 1, padding='valid', activation='relu', input_shape = (163, 10), use_bias= True)(pool1R2)
    pool2R2 = keras.layers.AveragePooling1D(pool_size=37, strides= 21, padding='valid')(conv2R2)
    conv3R2 = keras.layers.Conv1D(60, kernel_size= 1, strides= 1, padding='valid', activation='relu', input_shape = (35, 7), use_bias= True)(pool2R2)
    pool3R2 = keras.layers.GlobalAveragePooling1D()(conv3R2)
    flat2 =  keras.layers.Flatten()(pool3R2)

    conv1R3 = keras.layers.Conv1D(10, kernel_size= 9, strides= 1, padding='same', activation='relu', input_shape = (497, 16), use_bias= True)(uniqueInput)
    pool1R3 = keras.layers.AveragePooling1D(pool_size=11, strides= 3, padding='valid')(conv1R3)
    conv2R3 = keras.layers.Conv1D(35, kernel_size= 1, strides= 1, padding='valid', activation='relu', input_shape = (163, 10), use_bias= True)(pool1R3)
    pool2R3 = keras.layers.AveragePooling1D(pool_size=37, strides= 21, padding='valid')(conv2R3)
    conv3R3 = keras.layers.Conv1D(60, kernel_size= 1, strides= 1, padding='valid', activation='relu', input_shape = (35, 7), use_bias= True)(pool2R3)
    pool3R3 = keras.layers.GlobalAveragePooling1D()(conv3R3)
    flat3 =  keras.layers.Flatten()(pool3R3)

    concatenated_filters = keras.layers.concatenate([flat0, flat1, flat2, flat3])


    dense1 = keras.layers.Dense(100, activation=keras.layers.LeakyReLU(alpha=0.3))(concatenated_filters)  #keras.layers.LeakyReLU(alpha=0.3)
    dense2 = keras.layers.Dense(32, activation=keras.layers.LeakyReLU(alpha=0.3))(dense1)
    #dense3 = keras.layers.Dense(10, activation=keras.layers.LeakyReLU(alpha=0.3))(dense2)
    outpu1 = keras.layers.Dense(1, activation='sigmoid')(dense2)

    model = keras.Model(inputs= uniqueInput, outputs=outpu1)

    model.compile(loss=keras.losses.BinaryCrossentropy(reduction='sum_over_batch_size'), 
              metrics=['accuracy']
            , optimizer= keras.optimizers.Adam(learning_rate=0.001))
    
    model.fit(input_train, output_train, epochs= 15, batch_size = 32, 
                     validation_data=(input_validation, output_validation), shuffle= True)
    


In [3]:
raw_data_set1 = carregar_pickle('dataset_H3_W3S1.pkl')
input_full = raw_data_set1['inputs']
output_full = raw_data_set1['outputs']
sequences_full = raw_data_set1['sequences']

input_train, input_tv, output_train, output_tv, sequence_train, sequence_tv = train_test_split(input_full, output_full, 
sequences_full, train_size=0.70 , shuffle= True, random_state = 121);

input_validation, input_test, output_validation, output_test, sequence_validation, sequence_test = train_test_split(input_tv, output_tv, 
sequence_tv, train_size=0.5 , shuffle= False);

In [4]:

trainning_function(input_train, output_train, input_validation, output_validation)

Epoch 1/15
328/328 [==============================] - 58s 166ms/step - loss: 0.5510 - accuracy: 0.7110 - val_loss: 0.4603 - val_accuracy: 0.7901
Epoch 2/15
328/328 [==============================] - 51s 156ms/step - loss: 0.4633 - accuracy: 0.7902 - val_loss: 0.4449 - val_accuracy: 0.7946
Epoch 3/15
328/328 [==============================] - 51s 155ms/step - loss: 0.4358 - accuracy: 0.8062 - val_loss: 0.4092 - val_accuracy: 0.8160
Epoch 4/15
328/328 [==============================] - 51s 155ms/step - loss: 0.4348 - accuracy: 0.8060 - val_loss: 0.4052 - val_accuracy: 0.8347
Epoch 5/15
328/328 [==============================] - 50s 153ms/step - loss: 0.4075 - accuracy: 0.8233 - val_loss: 0.3919 - val_accuracy: 0.8329
Epoch 6/15
328/328 [==============================] - 56s 172ms/step - loss: 0.4061 - accuracy: 0.8260 - val_loss: 0.3822 - val_accuracy: 0.8396
Epoch 7/15
328/328 [==============================] - 51s 156ms/step - loss: 0.3971 - accuracy: 0.8259 - val_loss: 0.3861 - val_ac